# Data

In [13]:
import pandas as pd

In [14]:
name_file="./data/complete_davinci.csv"

df_initial = pd.read_csv(name_file)
concepts=list(df_initial.Program.unique())

In [15]:
name_file="./data/mt_setting.csv"

df = pd.read_csv(name_file,sep = ";")
size_comp = {1: "VL", 2: "VL",3: "L", 4: "L", 5: "H", 6: "H", 7: "VH", 8: "VH"}
for k,v in size_comp.items():
    df.loc[df.Length==k,"complexity"]=v
print(df.shape)
df=df[df["Program"].isin(concepts)]
print(df.shape)
df.reset_index(inplace=True)

(5062, 7)
(292, 7)


In [16]:
df

,index,Program,Length,WS,AS I,AS II,Test,complexity
0,0,+o,2,"[('', '0')]","[('01', '1'), ('01001', '1')]","[('1', ''), ('10', ''), ('1001', '')]","[('01010', '1'), ('11', ''), ('0', '1'), ('00'...",VL
1,1,-o,2,"[('', '1')]","[('01', ''), ('1', '0')]","[('1001', '0'), ('10', '0'), ('11001', '0')]","[('11', '0'), ('001', ''), ('0', ''), ('11010'...",VL
2,2,+oo,3,"[('', '00')]","[('1011', ''), ('1100', '')]","[('11', ''), ('011', '11'), ('1101', '')]","[('1110', ''), ('01011', '11'), ('1', ''), ('1...",L
3,3,+o+o,4,"[('', '01')]","[('010', '1'), ('0', '1')]","[('011', '1'), ('1', ''), ('100', '')]","[('10010', ''), ('10', ''), ('10011', ''), ('1...",L
4,4,-o-o,4,"[('', '10')]","[('1011', '0'), ('1', '0')]","[('01011', ''), ('11', '0'), ('011', '')]","[('01100', ''), ('100', '0'), ('0', ''), ('011...",L
...,...,...,...,...,...,...,...,...
287,4526,>[<]>-o,7,"[('0', '1'), ('00', ''), ('10', '0')]","[('11', '0'), ('01', '')]","[('1111', '0'), ('111', '0'), ('010', '')]","[('', '1'), ('011', ''), ('1', '1'), ('11111',...",VH
288,4752,[o>>]<+o,8,"[('', '0'), ('01', '0'), ('1', '10')]","[('010', '000'), ('1010', '111')]","[('011', '010'), ('0', '00'), ('1011', '11')]","[('11', '1'), ('100', '100'), ('00', '01'), ('...",VH
289,4761,>+o<[-o],8,"[('', '0'), ('00', '1'), ('1', '00')]","[('10', '10'), ('001', '1')]","[('1001', '10'), ('1010', '10'), ('1011', '10')]","[('1100', ''), ('01001', ''), ('0', '0'), ('01...",VH
290,4827,-[->]<o,7,"[('', '0'), ('0', ''), ('1', ''), ('11', '0')]","[('0110', ''), ('0111', '')]","[('110', ''), ('10', ''), ('00', '')]","[('01', ''), ('10110', ''), ('10111', '0'), ('...",VH


# Execution

In [17]:
from collections import Counter
import random


def get_output_from_list(list):
    outputs=[]
    for in_out in list:
        outputs.append(in_out[1])
    
    return outputs

def majority_strategy(outputs):
    counter=Counter(outputs)
    
    return counter.most_common(1)[0][0]

def random_strategy(output):
    num_sizes = 6 #empty string (size 0) + sizes from 1 to 5 = 6

    accuracy = 0
    
    size = len(output)
    probability_per_size=1/num_sizes
    accuracy_per_output=probability_per_size/ (2**size)
    return accuracy_per_output

In [18]:
import ast

majority_results_by_concept={}
random_results={}

for row_idx,row in df.iterrows():
    program=row.Program
    ws=row.WS
    ws=ast.literal_eval(ws)
    as1=row["AS I"]
    as1=ast.literal_eval(as1)
    as2=row["AS II"]
    as2=ast.literal_eval(as2)
    test=row.Test
    test=ast.literal_eval(test)
    complexity=row.complexity

    outputs_ws=get_output_from_list(ws)
    outputs_as1=get_output_from_list(as1)
    outputs_as2=get_output_from_list(as2)

    majority_output_ws=majority_strategy(outputs_ws)
    majority_output_as1=majority_strategy(outputs_as2)
    majority_output_as2=majority_strategy(outputs_as2)

    for i,question_in_test in enumerate(test):
        # Note there was an error in the original code, where the following line retrieved the index 0 (the input) 
        # rather than the index 1 (the output). Therefore, results on Table 20 of the Supplementary Material are incorrect.
        # The error was found after the final version was submitted, reason why these results are not reflected in the published 
        # document. 
        output_test=question_in_test[1]
        majority_result_ws=output_test==majority_output_ws
        majority_result_as1=output_test==majority_output_as1
        majority_result_as2=output_test==majority_output_as2

        majority_results_by_concept[program+"_"+"WS"+"_"+str(i)+"_"+complexity]=majority_result_ws
        majority_results_by_concept[program+"_"+"AS1"+"_"+str(i)+"_"+complexity]=majority_result_as1
        majority_results_by_concept[program+"_"+"AS2"+"_"+str(i)+"_"+complexity]=majority_result_as2

        random_choice=random_strategy(output_test)
        random_result=random_choice
        random_results[program+"_random"+"_"+str(i)+"_"+complexity]=random_result

# Majority Baseline 

In [19]:
import numpy as np
programs=[]
batches=[]
idx_tests=[]
values=[]
complexities=[]
for key,value in majority_results_by_concept.items():
    program,batch,idx_test,complexity=key.split("_")
    programs.append(program)
    batches.append(batch)
    idx_tests.append(idx_test)
    values.append(value)
    complexities.append(complexity)
df_result=pd.DataFrame(data={
    "Program":programs,
    "Batch":batches,
    "idx_test":idx_tests,
    "value":values,
    "complexity":complexities

})

In [20]:
df_result

,Program,Batch,idx_test,value,complexity
0,+o,WS,0,False,VL
1,+o,AS1,0,False,VL
2,+o,AS2,0,False,VL
3,+o,WS,1,False,VL
4,+o,AS1,1,True,VL
...,...,...,...,...,...
4375,+[-<]>+o,AS1,3,False,VH
4376,+[-<]>+o,AS2,3,False,VH
4377,+[-<]>+o,WS,4,True,VH
4378,+[-<]>+o,AS1,4,True,VH


In [21]:
df_result.groupby(["Batch","complexity"]).mean()

value
Batch complexity          
AS1   H           0.400000
      L           0.506667
      VH          0.424324
      VL          0.266667
AS2   H           0.400000
      L           0.506667
      VH          0.424324
      VL          0.266667
WS    H           0.206250
      L           0.324444
      VH          0.200000
      VL          0.200000

# Random Baseline

In [22]:
import numpy as np
programs=[]
batches=[]
idx_tests=[]
values=[]
complexities=[]
for key,value in random_results.items():
    program,batch,idx_test,complexity=key.split("_")
    programs.append(program)
    batches.append(batch)
    idx_tests.append(idx_test)
    values.append(value)
    complexities.append(complexity)
df_result=pd.DataFrame(data={
    "Program":programs,
    "Batch":batches,
    "idx_test":idx_tests,
    "value":values,
    "complexity":complexities

})

In [23]:
df_result

,Program,Batch,idx_test,value,complexity
0,+o,random,0,0.083333,VL
1,+o,random,1,0.166667,VL
2,+o,random,2,0.083333,VL
3,+o,random,3,0.083333,VL
4,+o,random,4,0.083333,VL
...,...,...,...,...,...
1455,+[-<]>+o,random,0,0.166667,VH
1456,+[-<]>+o,random,1,0.166667,VH
1457,+[-<]>+o,random,2,0.166667,VH
1458,+[-<]>+o,random,3,0.166667,VH


In [24]:
df_result.groupby(["Batch","complexity"]).mean()

value
Batch  complexity          
random H           0.077599
       L           0.075486
       VH          0.086400
       VL          0.100000